# Protein MD Setup tutorial using BioExcel Building Blocks (biobb) with remote GROMACS execution
**Based on the official GROMACS tutorial:** [http://www.mdtutorials.com/gmx/lysozyme/index.html](http://www.mdtutorials.com/gmx/lysozyme/index.html)
***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Lysozyme** protein (PDB code 1AKI). 
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_md](https://github.com/bioexcel/biobb_md): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup.git
cd biobb_wf_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_MDsetup_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_md_setup/notebooks/biobb_MDsetup_tutorial.ipynb
  ``` 

***
## Pipeline steps
 0. Setting up remote access
 0.1. Getting new credentials
 0.2. Installing credentials on host
 0.3. Setting host queue 
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create Solvent Box](#box)
 6. [Fill the Box with Water Molecules](#water)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min) (local)
 9. [Equilibrate the System (NVT)](#nvt) (remote)
 10. [Equilibrate the System (NPT)](#npt) (remote)
 11. [Free Molecular Dynamics Simulation](#free) (remote)
 12. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 13. [Output Files](#output)
 14. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


In [1]:
%load_ext autoreload
%autoreload 2

<a id="credentials"></a>
## Setting up remote access

Remote access uses standard ssh/sftp sessions. 
A specific public/private key pair will be generated (optional)

In [2]:
host = 'sl1.bsc.es'
userid = 'bsc23860'
host_config_path = '../conf/BSC_Starlife.json'

### Generate SSH keys and store locally
Skip to use user's credentials

In [3]:
#keys_file = 'bsc23860@mn1.keys'

#from biobb_remote.ssh_credentials import SSHCredentials

#credentials = SSHCredentials(
#    host=host, userid=userid, generate_key=True, look_for_keys=False
#)
#credentials.save(keys_file)

### Get generated keys

In [4]:
#credentials.get_private_key()

In [5]:
#credentials.get_public_key()

In [6]:
#credentials.sftp=None

Public key should to included in .ssh/authorized_keys, either manually or using install_host_auth (requires user's own ssh credentials)

In [7]:
#backup_file_ext = 'bck'
#credentials.install_host_auth(backup_file_ext)

Let's recover the keys from the local file. Useful to reuse previous sessions. 

In [8]:
#new_credentials = SSHCredentials()
#new_credentials.load_from_file(keys_file)
#new_credentials.check_host_auth()

Setting the connection to host queueing system (SLURM). 

local_path is a local working directory, should be created already. 

remote_path is a base path in the remote computer, will be created when necessary. remote_path will contain a different directory for each instance of the task manager created

task_data_path keeps a local copy of task manager status allowing to recover interrupted sessions

In [9]:
host = 'sl1.bsc.es'
userid = 'bsc23860'
local_path = '../test/test_wdir'
remote_path = 'scratch/test_biobb'
task_data_path = '../test/task_sl1_data.json'
# queue settings are bundled in Slurm class according to the options 
# on the remote computer
# modules are predefined bundles of HPC modules to be loaded.
queue_settings = 'default'
modules = ['gromacs']
conda_env = None

In [10]:
from biobb_remote.slurm import Slurm

##Option 1: Adding Biobb credentials set previously
#slurm = Slurm()
#slurm.set_credentials(credentials)
#slurm.load_host_config(host_config_path)
#slurm.save(task_data_path)

##Option 2: Using user's own credentials
slurm = Slurm(host=host, userid=userid, look_for_keys=True)
slurm.load_host_config(host_config_path)
slurm.save(task_data_path)

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [11]:
import nglview
import ipywidgets

pdbCode = "1AKI"

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [15]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import Pdb

# Create properties dict and inputs/outputs
downloaded_pdb = local_path + '/' + pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode
}

#Create and launch bb
Pdb(output_pdb_path=downloaded_pdb,
    properties=prop).launch()

2020-10-08 15:56:28,626 [MainThread  ] [INFO ]  Downloading: 1aki from: https://files.rcsb.org/download/1aki.pdb
2020-10-08 15:56:29,280 [MainThread  ] [INFO ]  Writting pdb to: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_wdir/1AKI.pdb
2020-10-08 15:56:29,281 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [ ]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.

***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [16]:
# Check & Fix PDB
# Import module
from biobb_model.model.fix_side_chain import FixSideChain

# Create prop dict and inputs/outputs
fixed_pdb = local_path + '/' + pdbCode + '_fixed.pdb'

# Create and launch bb
FixSideChain(input_pdb_path=downloaded_pdb, 
             output_pdb_path=fixed_pdb).launch()

2020-10-08 15:56:34,652 [MainThread  ] [INFO ]  check_structure -i ../test/test_wdir/1AKI.pdb -o ../test/test_wdir/1AKI_fixed.pdb --force_save fixside --fix ALL

2020-10-08 15:56:34,653 [MainThread  ] [INFO ]  Exit code 0

2020-10-08 15:56:34,654 [MainThread  ] [INFO ]  ===============================================================================
=                   MDWeb structure checking utility v1.1                     =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure ../test/test_wdir/1AKI.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Final Num. models: 1
Fina

0

### Visualizing 3D structure
Visualizing the fixed **PDB structure** using **NGL**. In this particular example, the checking step didn't find any issue to be solved, so there is no difference between the original structure and the fixed one.   

In [ ]:
# Show protein
view = nglview.show_structure_file(fixed_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_md.gromacs.pdb2gmx**
***

In [17]:
# Create system topology
# Import module
from biobb_md.gromacs.pdb2gmx import Pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = local_path + '/' + pdbCode+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = local_path + '/' + pdbCode+'_pdb2gmx_top.zip'

# Create and launch bb
Pdb2gmx(input_pdb_path=fixed_pdb, 
        output_gro_path=output_pdb2gmx_gro, 
        output_top_zip_path=output_pdb2gmx_top_zip).launch()

2020-10-08 15:56:37,785 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2020-10-08 15:56:37,786 [MainThread  ] [INFO ]  Not using any container
2020-10-08 15:56:38,082 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f ../test/test_wdir/1AKI_fixed.pdb -o ../test/test_wdir/1AKI_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2020-10-08 15:56:38,083 [MainThread  ] [INFO ]  Exit code 0

2020-10-08 15:56:38,086 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading ../test/test_wdir/1AKI_fixed.pdb...
Read '', 1001 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 129 residues with 1001 atoms

  chain  #res #atoms
  1 'A'   129   1001  

Reading residue database... (Amber99sb-ildn)

0

### Visualizing 3D structure
Visualizing the generated **GRO structure** using **NGL**. Note that **hydrogen atoms** were added to the structure by the **pdb2gmx GROMACS tool** when generating the **topology**.    

In [ ]:
# Show protein
view = nglview.show_structure_file(output_pdb2gmx_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **cubic box** is used to define the unit cell, with a **distance from the protein to the box edge of 1.0 nm**. The protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_md.gromacs.editconf** 
***

In [18]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import Editconf

# Create prop dict and inputs/outputs
output_editconf_gro = local_path + '/' + pdbCode+'_editconf.gro'

prop = {
    'box_type': 'cubic',
    'distance_to_molecule': 1.0
}

#Create and launch bb
Editconf(input_gro_path=output_pdb2gmx_gro, 
         output_gro_path=output_editconf_gro,
         properties=prop).launch()

2020-10-08 15:56:41,119 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2020-10-08 15:56:41,120 [MainThread  ] [INFO ]  Centering molecule in the box.
2020-10-08 15:56:41,121 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2020-10-08 15:56:41,122 [MainThread  ] [INFO ]  Box type: cubic
2020-10-08 15:56:41,122 [MainThread  ] [INFO ]  Not using any container
2020-10-08 15:56:41,154 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f ../test/test_wdir/1AKI_pdb2gmx.gro -o ../test/test_wdir/1AKI_editconf.gro -d 1.0 -bt cubic -c

2020-10-08 15:56:41,155 [MainThread  ] [INFO ]  Exit code 0

2020-10-08 15:56:41,155 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 1960 atoms
Volume: 55.8378 nm^3, corresponds to roughly 25100 electrons
No velocities found
    system size :  3.817  4.234  3.454 (nm)
    diameter    :  5.010               (nm)
    center      :  2.781  2.488  0.017 (

0

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein structure system** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_md.gromacs.solvate** 
***

In [19]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import Solvate

# Create prop dict and inputs/outputs
output_solvate_gro = local_path + '/' + pdbCode+'_solvate.gro'
output_solvate_top_zip = local_path + '/' + pdbCode+'_solvate_top.zip'

# Create and launch bb
Solvate(input_solute_gro_path=output_editconf_gro, 
        output_gro_path=output_solvate_gro, 
        input_top_zip_path=output_pdb2gmx_top_zip, 
        output_top_zip_path=output_solvate_top_zip).launch()

2020-10-08 15:56:43,615 [MainThread  ] [INFO ]  GROMACS Solvate 20191 version detected
2020-10-08 15:56:43,618 [MainThread  ] [INFO ]  Extracting: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_wdir/1AKI_pdb2gmx_top.zip
2020-10-08 15:56:43,619 [MainThread  ] [INFO ]  to:
2020-10-08 15:56:43,623 [MainThread  ] [INFO ]  ['ca10627b-801b-49d6-b63f-458c98b6daf1/p2g.top', 'ca10627b-801b-49d6-b63f-458c98b6daf1/posre.itp']
2020-10-08 15:56:43,628 [MainThread  ] [INFO ]  Unzipping: 
2020-10-08 15:56:43,628 [MainThread  ] [INFO ]  ../test/test_wdir/1AKI_pdb2gmx_top.zip
2020-10-08 15:56:43,629 [MainThread  ] [INFO ]  To: 
2020-10-08 15:56:43,630 [MainThread  ] [INFO ]  ca10627b-801b-49d6-b63f-458c98b6daf1/p2g.top
2020-10-08 15:56:43,630 [MainThread  ] [INFO ]  ca10627b-801b-49d6-b63f-458c98b6daf1/posre.itp
2020-10-08 15:56:43,631 [MainThread  ] [INFO ]  Not using any container
2020-10-08 15:56:44,044 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright solvate -cp ../test/test_wdir/1AKI_ed

0

### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** using **NGL**.<br> Note the **cubic box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the cube.

In [ ]:
# Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein structure** charge
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_md.gromacs.genion** 
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation
A simple **energy minimization** molecular dynamics parameters (mdp) properties will be used to generate the portable binary run file for **ion generation**, although **any legitimate combination of parameters** could be used in this step.

In [20]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import Grompp

# Create prop dict and inputs/outputs
output_gppion_tpr = local_path + '/' + pdbCode+'_gppion.tpr'
prop = {
    'mdp':{
        'type': 'minimization'
    }
}

# Create and launch bb
Grompp(input_gro_path=output_solvate_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,  
       properties=prop).launch()

2020-10-08 15:56:48,569 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2020-10-08 15:56:48,572 [MainThread  ] [INFO ]  Extracting: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_wdir/1AKI_solvate_top.zip
2020-10-08 15:56:48,573 [MainThread  ] [INFO ]  to:
2020-10-08 15:56:48,573 [MainThread  ] [INFO ]  ['da08f3d1-18c0-46a9-8905-f3039f5ce17c/p2g.top', 'da08f3d1-18c0-46a9-8905-f3039f5ce17c/posre.itp']
2020-10-08 15:56:48,574 [MainThread  ] [INFO ]  Unzipping: 
2020-10-08 15:56:48,574 [MainThread  ] [INFO ]  ../test/test_wdir/1AKI_solvate_top.zip
2020-10-08 15:56:48,579 [MainThread  ] [INFO ]  To: 
2020-10-08 15:56:48,583 [MainThread  ] [INFO ]  da08f3d1-18c0-46a9-8905-f3039f5ce17c/p2g.top
2020-10-08 15:56:48,584 [MainThread  ] [INFO ]  da08f3d1-18c0-46a9-8905-f3039f5ce17c/posre.itp
2020-10-08 15:56:48,585 [MainThread  ] [INFO ]  Will run a minimization md of 5000 steps
2020-10-08 15:56:48,585 [MainThread  ] [INFO ]  Not using any container
2020-10-08 15:56:48,882 

0

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [21]:
# Genion: Adding ions to neutralize the system
from biobb_md.gromacs.genion import Genion

# Create prop dict and inputs/outputs
output_genion_gro = local_path + '/' + pdbCode+'_genion.gro'
output_genion_top_zip = local_path + '/' + pdbCode+'_genion_top.zip'
prop={
    'neutral':True
}

# Create and launch bb
Genion(input_tpr_path=output_gppion_tpr, 
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_top_zip_path=output_genion_top_zip, 
       properties=prop).launch()

2020-10-08 15:56:51,128 [MainThread  ] [INFO ]  GROMACS Genion 20191 version detected
2020-10-08 15:56:51,131 [MainThread  ] [INFO ]  Extracting: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_wdir/1AKI_solvate_top.zip
2020-10-08 15:56:51,131 [MainThread  ] [INFO ]  to:
2020-10-08 15:56:51,132 [MainThread  ] [INFO ]  ['c321a1ab-7bed-4db5-bc16-9992ec9d17e2/p2g.top', 'c321a1ab-7bed-4db5-bc16-9992ec9d17e2/posre.itp']
2020-10-08 15:56:51,133 [MainThread  ] [INFO ]  Unzipping: 
2020-10-08 15:56:51,133 [MainThread  ] [INFO ]  ../test/test_wdir/1AKI_solvate_top.zip
2020-10-08 15:56:51,134 [MainThread  ] [INFO ]  To: 
2020-10-08 15:56:51,136 [MainThread  ] [INFO ]  c321a1ab-7bed-4db5-bc16-9992ec9d17e2/p2g.top
2020-10-08 15:56:51,137 [MainThread  ] [INFO ]  c321a1ab-7bed-4db5-bc16-9992ec9d17e2/posre.itp
2020-10-08 15:56:51,138 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2020-10-08 15:56:51,138 [MainThread  ] [INFO ]  Not using any container
2020-10-08 15:56:51,246 

0

### Visualizing 3D structure
Visualizing the **neutralized protein system** with the newly added **ions** using **NGL**

In [ ]:
# Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
The **minimization** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **energy minimization**:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01 ; Minimization step size (nm)
-  nsteps      = 50000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, but the **maximum force** is placed at **500 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5,000 steps**. 

In [22]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp

# Create prop dict and inputs/outputs
output_gppmin_tpr = local_path + '/' + pdbCode+'_gppmin.tpr'
prop = {
    'mdp':{
        'type': 'minimization',
        'emtol':'500',
        'nsteps':'5000'
    }
}

# Create and launch bb
Grompp(input_gro_path=output_genion_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppmin_tpr,  
       properties=prop).launch()

2020-10-08 15:56:54,793 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2020-10-08 15:56:54,796 [MainThread  ] [INFO ]  Extracting: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_wdir/1AKI_genion_top.zip
2020-10-08 15:56:54,802 [MainThread  ] [INFO ]  to:
2020-10-08 15:56:54,804 [MainThread  ] [INFO ]  ['979348ff-102e-4c89-b581-837f969f7689/p2g.top', '979348ff-102e-4c89-b581-837f969f7689/posre.itp']
2020-10-08 15:56:54,805 [MainThread  ] [INFO ]  Unzipping: 
2020-10-08 15:56:54,808 [MainThread  ] [INFO ]  ../test/test_wdir/1AKI_genion_top.zip
2020-10-08 15:56:54,809 [MainThread  ] [INFO ]  To: 
2020-10-08 15:56:54,811 [MainThread  ] [INFO ]  979348ff-102e-4c89-b581-837f969f7689/p2g.top
2020-10-08 15:56:54,815 [MainThread  ] [INFO ]  979348ff-102e-4c89-b581-837f969f7689/posre.itp
2020-10-08 15:56:54,818 [MainThread  ] [INFO ]  Will run a minimization md of 5000 steps
2020-10-08 15:56:54,819 [MainThread  ] [INFO ]  Not using any container
2020-10-08 15:56:55,121 [M

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization (remote)
Running **energy minimization** using the **tpr file** generated in the previous step. 

Setting local data and uploading files to remote. 

In [23]:
slurm.set_local_data_bundle(local_path, add_files=False)
slurm.task_data['local_data_bundle'].add_file(output_gppmin_tpr)
slurm.send_input_data(remote_path, overwrite=False)

sending_file: ../test/test_wdir/1AKI_gppmin.tpr -> scratch/test_biobb/biobb_4da3777e-0b7c-4691-92c4-bc4f8939bf4c/1AKI_gppmin.tpr


In [26]:
slurm.load_host_config(host_config_path)

In [39]:
# Mdrun: Running minimization

# Create prop dict and inputs/outputs
output_min_trr = pdbCode+'_min.trr'
output_min_gro = pdbCode+'_min.gro'
output_min_edr = pdbCode+'_min.edr'
output_min_log = pdbCode+'_min.log'

files = {
    '-s' : pdbCode + '_gppmin.tpr', 
    '-o' : output_min_trr, 
    '-c' : output_min_gro, 
    '-e' : output_min_edr, 
    '-g' : output_min_log
}

#num_threads = 64


# cmd_setttings
# Python dict
cmd_set = {
#    'use_gpu' : True
#    'dlb': True,
}

command = 'srun gmx_mpi mdrun'
# properties
# Python dict
# patching queue settings
patch = slurm.prep_auto_settings(nodes=4, cpus_per_task=6)
slurm.set_custom_settings(patch=patch, clean=True)
patch = {'time':'1:00:00'}
slurm.set_custom_settings(ref_setting='custom', patch=patch)

# for a single biobb execution on a slurm job
# Alternatively, a file containing a more complex script can be loaded from disk

slurm.submit(
    queue_settings='custom',
    modules=modules,
    conda_env=conda_env,
    local_run_script=slurm.get_remote_comm_line(command, files, cmd_settings=cmd_set)
)

Submitted job 8609225


Waiting for job completion and saving status. Poll time is in seconds.

In [40]:
slurm.check_job(poll_time=5)
slurm.save(task_data_path)

 Job 8609225 is Finished


Getting logs

In [41]:
print(slurm.get_logs())

 Job 8609225 is Finished
('', 'Set INTEL compilers as MPI wrappers backend \nload impi/2018.3 (PATH, MANPATH, LD_LIBRARY_PATH) \nload GROMACS/2019.1 (PATH, MANPATH, LD_LIBRARY_PATH, LIBRARY_PATH,\nPKG_CONFIG_PATH, C_INCLUDE_PATH, CPLUS_INCLUDE_PATH) \n                       :-) GROMACS - gmx mdrun, 2019 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen\n    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     \n  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  \n  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis\n    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    \n  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   \n    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   \n    Sand

Recovering output files to local_path

In [42]:
slurm.get_output_data(overwrite=False)

getting_file: scratch/test_biobb/biobb_4da3777e-0b7c-4691-92c4-bc4f8939bf4c/#1AKI_min.gro.1# -> ../test/test_wdir/#1AKI_min.gro.1#


<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [43]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import GMXEnergy

# Create prop dict and inputs/outputs
output_min_edr = local_path + "/" + pdbCode + "_min.edr"
output_min_ene_xvg = local_path + "/" + pdbCode+'_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
GMXEnergy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop).launch()

2020-10-08 18:15:08,163 [MainThread  ] [INFO ]  Not using any container
2020-10-08 18:15:08,187 [MainThread  ] [INFO ]  gmx energy -f /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/notebook/../test/test_wdir/1AKI_min.edr -o ../test/test_wdir/1AKI_min_ene.xvg -xvg none < fa1e1b52-6630-418f-b5c2-97fa70280859/instructions.in

2020-10-08 18:15:08,192 [MainThread  ] [INFO ]  Exit code 0

2020-10-08 18:15:08,193 [MainThread  ] [INFO ]  
Statistics over 2105 steps [ 0.0000 through 2104.0000 ps ], 1 data sets
All statistics are over 1666 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -582898       9300    23831.8   -61263.4  (kJ/mol)

2020-10-08 18:15:08,193 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     He

0

In [44]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein system** in **NVT ensemble** (constant Number of particles, Volume and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNVTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNVTStep2): Equilibrate the **protein system** with **NVT** ensemble.
- [Step 3](#eqNVTStep3): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 
***
**Building Blocks** used:
- [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
- [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
- [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNVTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NVT)
The **nvt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NVT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

In this particular example, the default parameters will be used: **md** integrator algorithm, a **step size** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a temperature of **300K**.

*Please note that for the sake of time this tutorial is only running 10ps of NVT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/06_equil.html) the simulated time was 100ps.*

In [45]:
# Grompp: Creating portable binary run file for NVT Equilibration
from biobb_md.gromacs.grompp import Grompp

# Create prop dict and inputs/outputs
input_min_gro=local_path + "/" + pdbCode + '_min.gro' 
input_genion_top_zip = local_path + "/" + pdbCode + '_genion_top.zip'

output_gppnvt_tpr = local_path + "/" + pdbCode+'_gppnvt.tpr'

prop = {
    'mdp':{
        'type': 'nvt',
        'nsteps': 5000,
        'dt': 0.002,
        'define': '-DPOSRES',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    }
}

# Create and launch bb
Grompp(input_gro_path=input_min_gro, 
       input_top_zip_path=input_genion_top_zip, 
       output_tpr_path=output_gppnvt_tpr,  
       properties=prop).launch()

2020-10-08 18:15:16,399 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2020-10-08 18:15:16,403 [MainThread  ] [INFO ]  Extracting: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_wdir/1AKI_genion_top.zip
2020-10-08 18:15:16,404 [MainThread  ] [INFO ]  to:
2020-10-08 18:15:16,406 [MainThread  ] [INFO ]  ['88332082-1ae7-48ad-b623-992c3d311e28/p2g.top', '88332082-1ae7-48ad-b623-992c3d311e28/posre.itp']
2020-10-08 18:15:16,406 [MainThread  ] [INFO ]  Unzipping: 
2020-10-08 18:15:16,408 [MainThread  ] [INFO ]  ../test/test_wdir/1AKI_genion_top.zip
2020-10-08 18:15:16,409 [MainThread  ] [INFO ]  To: 
2020-10-08 18:15:16,413 [MainThread  ] [INFO ]  88332082-1ae7-48ad-b623-992c3d311e28/p2g.top
2020-10-08 18:15:16,415 [MainThread  ] [INFO ]  88332082-1ae7-48ad-b623-992c3d311e28/posre.itp
2020-10-08 18:15:16,417 [MainThread  ] [INFO ]  Will run a nvt md of 5000 steps
2020-10-08 18:15:16,418 [MainThread  ] [INFO ]  Not using any container
2020-10-08 18:15:16,821 [MainThread

0

Uploading new tpr file to remote

In [46]:
slurm.task_data['local_data_bundle'].add_file(output_gppnvt_tpr)
slurm.send_input_data(remote_path, overwrite= False)


sending_file: ../test/test_wdir/1AKI_gppnvt.tpr -> scratch/test_biobb/biobb_4da3777e-0b7c-4691-92c4-bc4f8939bf4c/1AKI_gppnvt.tpr


<a id="eqNVTStep2"></a>
### Step 2: Running NVT equilibration (remote)

In [57]:
# Mdrun: Running Equilibration NVT
python_import = 'from biobb_md.gromacs.mdrun import Mdrun'

# Create prop dict and inputs/outputs

input_gppnvt_tpr = pdbCode + '_gppnvt.tpr'

output_nvt_trr = pdbCode+'_nvt.trr'
output_nvt_gro = pdbCode+'_nvt.gro'
output_nvt_edr = pdbCode+'_nvt.edr'
output_nvt_log = pdbCode+'_nvt.log'
output_nvt_cpt = pdbCode+'_nvt.cpt'

files = {
    '-s' : input_gppnvt_tpr, 
    '-o' : output_nvt_trr, 
    '-c' : output_nvt_gro, 
    
    '-e' : output_nvt_edr, 
    '-g' : output_nvt_log
}

# cmd_setttings
# Python dict
cmd_set = {
#    'use_gpu' : True
    '-dlb': 'yes'
#    '-ntomp': num_threads
}

patch = slurm.prep_auto_settings(nodes=1, cpus_per_task=40)
slurm.set_custom_settings(patch=patch, clean=True)

patch = {'time':'1:00:00'}
slurm.set_custom_settings(ref_setting='custom', patch=patch)
print(slurm.host_config['qsettings']['custom'])
command = 'srun gmx_mpi mdrun'

# patching queue settings


#patch = {'qos' : 'debug','time':'1:00:00', 'cpus-per-task': num_threads}
#slurm.set_custom_settings(patch=patch)


slurm.submit(
    queue_settings = 'custom', 
    modules = modules, 
    local_run_script = slurm.get_remote_comm_line(command, files, cmd_settings=cmd_set),
    job_name = pdbCode + '_nvt'
)


{'ntasks': 1, 'cpus-per-task': 40, 'ntasks-per-node': 1, 'nodes': 1, 'time': '1:00:00'}
Submitted job 8618377


In [55]:
slurm.check_job(poll_time=5)
slurm.save(task_data_path)

0 Job 8616032 is Running
5 Job 8616032 is Running
10 Job 8616032 is Running
15 Job 8616032 is Closing
 Job 8616032 is Finished


In [ ]:
slurm.get_output_data(overwrite=False)

<a id="eqNVTStep3"></a>
### Step 3: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [ ]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration  
from biobb_analysis.gromacs.gmx_energy import GMXEnergy

# Create prop dict and inputs/outputs
input_nvt_edr = local_path + '/' + pdbCode + '_nvt.edr'
output_nvt_temp_xvg = local_path + '/' + pdbCode+'_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
GMXEnergy(input_energy_path=input_nvt_edr, 
          output_xvg_path=output_nvt_temp_xvg, 
          properties=prop).launch()

In [ ]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 5000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **5,000 equilibration steps** with the protein **heavy atoms restrained**, and a Parrinello-Rahman **pressure coupling** algorithm.

*Please note that for the sake of time this tutorial is only running 10ps of NPT equilibration, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/07_equil2.html) the simulated time was 100ps.*

In [ ]:
# Grompp: Creating portable binary run file for NPT System Equilibration
from biobb_md.gromacs.grompp import Grompp

# Create prop dict and inputs/outputs
input_nvt_gro = local_path + "/" + pdbCode + '_nvt.gro'
output_gppnpt_tpr = local_path + "/" + pdbCode+'_gppnpt.tpr'
prop = {
    'mdp':{
        'type': 'npt',
        'nsteps':'5000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    }
}

# Create and launch bb
Grompp(input_gro_path=input_nvt_gro, 
       input_top_zip_path=input_genion_top_zip, 
       output_tpr_path=output_gppnpt_tpr, 
       input_cpt_path=output_nvt_cpt,  
       properties=prop).launch()

In [ ]:
slurm.task_data['local_data_bundle'].add_file(output_gppnpt_tpr)
slurm.send_input_data(remote_path, overwrite= False)


<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [ ]:
# Mdrun: Running NPT System Equilibration
python_import = 'from biobb_md.gromacs.mdrun import Mdrun'

# Create prop dict and inputs/outputs
input_nvt_tpr  = pdbCode+'_gppnpt.tpr' 
output_npt_trr = pdbCode+'_npt.trr'
output_npt_gro = pdbCode+'_npt.gro'
output_npt_edr = pdbCode+'_npt.edr'
output_npt_log = pdbCode+'_npt.log'
output_npt_cpt = pdbCode+'_npt.cpt'

files = {
    '-s' : input_nvt_tpr, 
    '-o' :output_npt_trr, 
    '-c' :output_npt_gro, 
    '-e' :output_npt_edr, 
    '-g' :output_npt_log, 
    '-cpo' :output_npt_cpt
}
command = 'gmx_mpi mdrun'

cmd_set={}

slurm.submit(
    'custom', 
    modules, 
    slurm.get_remote_comm_line(command, files, cmd_settings=cmd_set)
)


In [ ]:
slurm.check_job(poll_time=5)
slurm.save(task_data_path)

In [ ]:
slurm.get_output_data(overwrite=False)

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [ ]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import GMXEnergy

# Create prop dict and inputs/outputs
input_npt_edr = local_path + "/" + pdbCode + '_npt.edr'
output_npt_pd_xvg = pdbCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
GMXEnergy(input_energy_path=input_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop).launch()

In [ ]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 500000

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **50,000 md steps** (100ps).

*Please note that for the sake of time this tutorial is only running 100ps of free MD, whereas in the [original example](http://www.mdtutorials.com/gmx/lysozyme/08_MD.html) the simulated time was 1ns (1000ps).*

In [ ]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp

# Create prop dict and inputs/outputs
input_npt_gro = local_path + '/' + pdbCode + '_npt.gro'
output_gppmd_tpr = local_path + '/' + pdbCode+'_gppmd.tpr'
prop = {
    'mdp':{
        'type': 'free',
        'nsteps':'500000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    }
}

# Create and launch bb
Grompp(input_gro_path=input_npt_gro, 
       input_top_zip_path=input_genion_top_zip, 
       output_tpr_path=output_gppmd_tpr, 
       input_cpt_path=output_npt_cpt, 
       properties=prop).launch()

In [ ]:
slurm.task_data['local_data_bundle'].add_file(output_gppmd_tpr)
slurm.send_input_data(remote_path, overwrite= False)


<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [ ]:
# Mdrun: Running NPT System Equilibration

# Create prop dict and inputs/outputs
input_npt_tpr  = pdbCode+'_gppmd.tpr' 
output_md_trr = pdbCode+'_md.trr'
output_md_gro = pdbCode+'_md.gro'
output_md_edr = pdbCode+'_md.edr'
output_md_log = pdbCode+'_md.log'
output_md_cpt = pdbCode+'_md.cpt'

files = {
    '-s' : input_npt_tpr, 
    '-o' :output_md_trr, 
    '-c' :output_md_gro, 
    '-e' :output_md_edr, 
    '-g' :output_md_log, 
    '-cpo' :output_md_cpt
}

command = 'gmx_mpi mdrun'

cmd_set={}

slurm.submit(
    'custom', 
    modules, 
    slurm.get_remote_comm_line(command, files, cmd_settings=cmd_set)
)


In [ ]:
slurm.check_job(poll_time=50)
slurm.save(task_data_path)

In [ ]:
slurm.get_output_data(overwrite=False)

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import GMXRms

# Create prop dict and inputs/outputs
input_gppmd_tpr = local_path  + '/' + pdbCode + '_gppmd.tpr'
input_md_trr = local_path  + '/' + pdbCode + '_md.trr'
output_rms_first = local_path  + '/' + pdbCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
GMXRms(input_structure_path=input_gppmd_tpr,
         input_traj_path=input_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop).launch()

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import GMXRms

# Create prop dict and inputs/outputs
input_gppmin_tpr = local_path  + '/' + pdbCode + '_gppmin.tpr'
input_traj_tpr = local_path  + '/' + pdbCode + '_md.trr'

output_rms_exp = pdbCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
GMXRms(input_structure_path=input_gppmin_tpr,
         input_traj_path=input_md_trr,
         output_xvg_path=output_rms_exp, 
          properties=prop).launch()

In [ ]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [ ]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import GMXRgyr

# Create prop dict and inputs/outputs
output_rgyr = local_path + '/' + pdbCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
GMXRgyr(input_structure_path=input_gppmin_tpr,
         input_traj_path=input_md_trr,
         output_xvg_path=output_rgyr, 
          properties=prop).launch()

In [ ]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
}

plotly.offline.iplot(fig)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [ ]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import GMXImage

# Create prop dict and inputs/outputs

output_imaged_traj = local_path + '/' + pdbCode+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
GMXImage(input_traj_path=input_md_trr,
         input_top_path=input_gppmd_tpr,
         output_traj_path=output_imaged_traj, 
          properties=prop).launch()

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [ ]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import GMXTrjConvStr

# Create prop dict and inputs/outputs
input_md_gro = local_path + '/' + pdbCode + '_md.gro'
output_dry_gro = local_path + '/' + pdbCode+'_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Create and launch bb
GMXTrjConvStr(input_structure_path=input_md_gro,
         input_top_path=input_gppmd_tpr,
         output_str_path=output_dry_gro, 
          properties=prop).launch()

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [ ]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

Clean remote files amd remove credentials


In [ ]:
#slurm.clean_remote()
#credentials.remove_host_auth()

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
